In [9]:
from __future__ import print_function

from google_api_service_getters.google_docs_service_getter import google_docs_service_getter

from google_docs_section_idx_getter import section_idx_getter

import json

import os

In [10]:
# menzies_bar_oos_test doc 

doc_id = "1Iyy3ltWTNkEk45fP5rD7efiXG1kJlmTYehauf100DqY"

service = google_docs_service_getter()

Retrieve the documents contents from the Docs service.

In [13]:
doc = service.documents().get(documentId=doc_id).execute()

# The content list has a length of 98.

doc_content = doc["body"]["content"]

section_idx_getter(doc_content, target_section="LOW BTB")

for i in range(len(doc["body"]["content"])):
    print(i)
    print(doc["body"]["content"][i])
    ['paragraph']['elements'][0]['textRun']['content']

print(doc["body"]["content"][-24]['paragraph']['elements'][0]['textRun']['content'])

doc["body"]["content"][-20]['paragraph']['elements'][0]['textRun']['content']="hello world!"

for i in range(10, len(doc["body"]["content"])):
    print(-i)
    print(doc["body"]["content"][-i]["paragraph"]['elements'][0]['textRun']['content'])

print(doc["body"]["content"][0])

<>:12: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:12: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?


The following is the idx values of each heading in the document content list, not their start and stop indexes
9 Notes
The following is the idx values of each heading in the document content list, not their start and stop indexes
15 Specials:
The following is the idx values of each heading in the document content list, not their start and stop indexes
23 OOS BTG
The following is the idx values of each heading in the document content list, not their start and stop indexes
27 LOW BTG
The following is the idx values of each heading in the document content list, not their start and stop indexes
47 OOS BTB
The following is the idx values of each heading in the document content list, not their start and stop indexes
59 LOW BTB
The following is the idx values of each heading in the document content list, not their start and stop indexes
85 TO CHANGE ON MENU
The following is the idx values of each heading in the document content list, not their start and stop indexes
87 VINTAGE CHANGES/LIST CH

/var/folders/mh/cknmc22s2zv5m4l777wwz1b00000gn/T/ipykernel_96802/4079541825.py:12: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  ['paragraph']['elements'][0]['textRun']['content']


TypeError: list indices must be integers or slices, not str

Identify which items in the content list do not contain the paragraph field.

In [14]:
content = doc["body"]["content"]

for count, i in enumerate(content):
    if "paragraph" not in i.keys():
        print(count)
        print(i)

# So items 0 and 9 specifically do not contain the "paragraph" field. Why? What does the 'content' field contain? We can understand this by observing the start and end indexes.

for i in range(2):
    print(i)
    print(json.dumps(content[i], indent=4))

0
{'endIndex': 1, 'sectionBreak': {'sectionStyle': {'columnSeparatorStyle': 'NONE', 'contentDirection': 'LEFT_TO_RIGHT', 'sectionType': 'CONTINUOUS'}}}
7
{'startIndex': 81, 'endIndex': 82, 'sectionBreak': {'sectionStyle': {'columnProperties': [{'width': {'magnitude': 207.637, 'unit': 'PT'}, 'paddingEnd': {'magnitude': 36, 'unit': 'PT'}}, {'width': {'magnitude': 207.637, 'unit': 'PT'}, 'paddingEnd': {'unit': 'PT'}}], 'columnSeparatorStyle': 'NONE', 'contentDirection': 'LEFT_TO_RIGHT', 'sectionType': 'CONTINUOUS'}}}
0
{
    "endIndex": 1,
    "sectionBreak": {
        "sectionStyle": {
            "columnSeparatorStyle": "NONE",
            "contentDirection": "LEFT_TO_RIGHT",
            "sectionType": "CONTINUOUS"
        }
    }
}
1
{
    "startIndex": 1,
    "endIndex": 29,
    "paragraph": {
        "elements": [
            {
                "startIndex": 1,
                "endIndex": 27,
                "textRun": {
                    "content": "17/01/2023 MENZIES BAR OOS",
   

The start and end indexes are as though all of the text characters (including whitespace) exist in a 1D array. Adding text to the end will increase the highest value of "endIndex". Inserting text in the middle will increase both the startIndex and endIndex for all items in the content list after the insertion point.

To find the section, we need to locate the headers. The headers are identified by the document.body.content.paragraph.paragraphStyle.namedStyleType field. The content of the headers is contained in the document.body.content.paragraph.elements[] list, and the start and end indexes can be found under document.body.content.startIndex and document.body.content.endIndex respectively. To find all the headers we can:



In [15]:
for i in range(len(content)):
    if "paragraph" in content[i].keys() and "HEADING" in content[i]["paragraph"]["paragraphStyle"]["namedStyleType"]:
        print(i)
        print(json.dumps(content[i], indent=4))

# The below block of code constructs a list of the section heading strings, stripping them of whitespace characters. repr() is used to print white space characters.

heading_dict = {}


9
{
    "startIndex": 83,
    "endIndex": 89,
    "paragraph": {
        "elements": [
            {
                "startIndex": 83,
                "endIndex": 88,
                "textRun": {
                    "content": "Notes",
                    "textStyle": {
                        "bold": true,
                        "underline": true,
                        "fontSize": {
                            "magnitude": 12,
                            "unit": "PT"
                        },
                        "weightedFontFamily": {
                            "fontFamily": "Times New Roman",
                            "weight": 400
                        }
                    }
                }
            },
            {
                "startIndex": 88,
                "endIndex": 89,
                "textRun": {
                    "content": "\n",
                    "textStyle": {}
                }
            }
        ],
        "paragraphStyle": {
            

In [16]:
# and if we copy the above loop, targeting the titles instead:

for i in range(len(doc_content)):
    if "paragraph" in doc_content[i].keys() and "HEADING" in doc_content[i]["paragraph"]["paragraphStyle"]["namedStyleType"]:
        
        heading = doc_content[i]["paragraph"]["elements"][0]["textRun"]["content"].strip()
        
        heading_dict[heading] = i

print(heading_dict)


{'Notes': 9, 'Specials:': 15, 'OOS BTG': 23, 'LOW BTG': 27, 'OOS BTB': 47, 'LOW BTB': 59, 'TO CHANGE ON MENU': 85, 'VINTAGE CHANGES/LIST CHANGES': 87, 'UPSTAIRS WINE': 90}


In [ ]:
# To control the text of a section, we need to know where it starts and where it finishes. A section begins at the index of the content list the heading is in, and ends at i-1 of the title following. Assembling a dictionary with "title" "content_idx" could be the way to go:

print(section_dict)



In [ ]:
# now we want to use the dict ot get the content idx then iterate through the content list printing the textRun content of each line IF they are not whitespace, until the next heading.

target_section = "LOW BTB"

start_idx = heading_dict[target_section]

In [ ]:
# Because I cannot easily iterate through the dict by 1 advancement, the complicated expresion below converts the keys() to a list then gets the index of the target section and puts that back into the keys() list to move 1 item forward, then get the content index of that line. Since we actually want the line before, we will subtract 1 from that value.

end_idx = heading_dict[
                        list(heading_dict.keys())
                            [
                                list(heading_dict.keys()).index(target_section)+1]
                                ]-1

print(end_idx)

In [ ]:
# remembering that the accessor for the text is of the form content[content_idx]["paragraph"]["elements"][0]["textRun"]["content"]. To make it easier, define a short function to iterate through the content_idx position while leaving the rest of the accessors as a template:

def content_text(content_idx):
    return doc_content[content_idx]["paragraph"]["elements"][0]["textRun"]["content"]

for i in range(start_idx, end_idx):
    if not content_text(i).isspace():
        print(content_text(i).strip())